# Mosaic AI Agent Framework: Author and deploy a tool-calling LangGraph agent

This notebook demonstrates how to author a LangGraph agent that's compatible with Mosaic AI Agent Framework features. In this notebook you learn to:
- Author a tool-calling LangGraph agent wrapped with `ChatAgent`
- Manually test the agent's output
- Evaluate the agent using Mosaic AI Agent Evaluation
- Log and deploy the agent

To learn more about authoring an agent using Mosaic AI Agent Framework, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/author-agent) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/create-chat-model)).

## Prerequisites

- Address all `TODO`s in this notebook.

In [0]:
%pip install -U -qqqq mlflow databricks-langchain databricks-agents uv langgraph==0.3.4  arize-otel openinference.instrumentation.langchain
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.



## Define the agent in code
Define the agent code in a single cell below. This lets you easily write the agent code to a local Python file, using the `%%writefile` magic command, for subsequent logging and deployment.

#### Agent tools
This agent code adds the built-in Unity Catalog function `system.ai.python_exec` to the agent. The agent code also includes commented-out sample code for adding a vector search index to perform unstructured data retrieval.

For more examples of tools to add to your agent, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/agent-tool) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/agent-tool))

#### Wrap the LangGraph agent using the `ChatAgent` interface

For compatibility with Databricks AI features, the `LangGraphChatAgent` class implements the `ChatAgent` interface to wrap the LangGraph agent. This example uses the provided convenience APIs [`ChatAgentState`](https://mlflow.org/docs/latest/python_api/mlflow.langchain.html#mlflow.langchain.chat_agent_langgraph.ChatAgentState) and [`ChatAgentToolNode`](https://mlflow.org/docs/latest/python_api/mlflow.langchain.html#mlflow.langchain.chat_agent_langgraph.ChatAgentToolNode) for ease of use.

Databricks recommends using `ChatAgent` as it simplifies authoring multi-turn conversational agents using an open source standard. See MLflow's [ChatAgent documentation](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent).



In [0]:
# Reading the secure keys from secrets
ARIZE_API_KEY = dbutils.secrets.get(scope="prasad_kona", key="ARIZE_API_KEY")
ARIZE_SPACE_ID = dbutils.secrets.get(scope="prasad_kona", key="ARIZE_SPACE_ID")

# setting as environment variables to be used by the chain
import os
os.environ["ARIZE_API_KEY"] = ARIZE_API_KEY
os.environ["ARIZE_SPACE_ID"] = ARIZE_SPACE_ID


In [0]:
%%writefile agent.py
from typing import Any, Generator, Optional, Sequence, Union

import mlflow
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    VectorSearchRetrieverTool,
)
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
import os
#mlflow.langchain.autolog()

import logging
logging.getLogger("openinference.instrumentation.langchain._tracer").setLevel(logging.CRITICAL)

############################################
# Arize Tracing Setup
############################################
#register tracer provider to send traces to Arize
from arize.otel import register

model_config = mlflow.models.ModelConfig(development_config="chain_config.yaml")

tracer_provider = register(
    space_id = os.getenv("ARIZE_SPACE_ID"),
    api_key = os.getenv("ARIZE_API_KEY"),
    project_name = model_config.get("ARIZE_PROJECT_NAME"),
    #log_to_console=True
)
# 1 line auto instrumentation
from openinference.instrumentation.langchain import LangChainInstrumentor
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)


############################################
# Define your LLM endpoint and system prompt
############################################
# TODO: Replace with your model serving endpoint
LLM_ENDPOINT_NAME = model_config.get("LLM_ENDPOINT_NAME") 
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

# TODO: Update with your system prompt
system_prompt = ""

###############################################################################
## Define tools for your agent, enabling it to retrieve data or take actions
## beyond text generation
## To create and see usage examples of more tools, see
## https://docs.databricks.com/en/generative-ai/agent-framework/agent-tool.html
###############################################################################
tools = []

# You can use UDFs in Unity Catalog as agent tools
# Below, we add the `system.ai.python_exec` UDF, which provides
# a python code interpreter tool to our agent
# You can also add local LangChain python tools. See https://python.langchain.com/docs/concepts/tools

# TODO: Add additional tools
uc_tool_names = ["system.ai.python_exec"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools.extend(uc_toolkit.tools)

# Use Databricks vector search indexes as tools
# See https://docs.databricks.com/en/generative-ai/agent-framework/unstructured-retrieval-tools.html
# for details

# TODO: Add vector search indexes
# vector_search_tools = [
#         VectorSearchRetrieverTool(
#         index_name="",
#         # filters="..."
#     )
# ]
# tools.extend(vector_search_tools)

#####################
## Define agent logic
#####################


def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[ToolNode, Sequence[BaseTool]],
    system_prompt: Optional[str] = None,
) -> CompiledGraph:
    model = model.bind_tools(tools)

    # Define the function that determines which node to go to
    def should_continue(state: ChatAgentState):
        messages = state["messages"]
        last_message = messages[-1]
        # If there are function calls, continue. else, end
        if last_message.get("tool_calls"):
            return "continue"
        else:
            return "end"

    if system_prompt:
        preprocessor = RunnableLambda(
            lambda state: [{"role": "system", "content": system_prompt}]
            + state["messages"]
        )
    else:
        preprocessor = RunnableLambda(lambda state: state["messages"])
    model_runnable = preprocessor | model

    def call_model(
        state: ChatAgentState,
        config: RunnableConfig,
    ):
        response = model_runnable.invoke(state, config)

        return {"messages": [response]}

    workflow = StateGraph(ChatAgentState)

    workflow.add_node("agent", RunnableLambda(call_model))
    workflow.add_node("tools", ChatAgentToolNode(tools))

    workflow.set_entry_point("agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools",
            "end": END,
        },
    )
    workflow.add_edge("tools", "agent")

    return workflow.compile()


class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        request = {"messages": self._convert_messages_to_dict(messages)}

        messages = []
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        request = {"messages": self._convert_messages_to_dict(messages)}
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg}) for msg in node_data["messages"]
                )


# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
mlflow.langchain.autolog()
agent = create_tool_calling_agent(llm, tools, system_prompt)
AGENT = LangGraphChatAgent(agent)
mlflow.models.set_model(AGENT)

Overwriting agent.py


## Test the agent

Interact with the agent to test its output and tool-calling abilities. Since this notebook called `mlflow.langchain.autolog()`, you can view the trace for each step the agent takes.

Replace this placeholder input with an appropriate domain-specific example for your agent.

In [0]:
dbutils.library.restartPython()

In [0]:
# Reading the secure keys from secrets
ARIZE_API_KEY = dbutils.secrets.get(scope="prasad_kona", key="ARIZE_API_KEY")
ARIZE_SPACE_ID = dbutils.secrets.get(scope="prasad_kona", key="ARIZE_SPACE_ID")

# setting as environment variables to be used by the chain
import os
os.environ["ARIZE_API_KEY"] = ARIZE_API_KEY
os.environ["ARIZE_SPACE_ID"] = ARIZE_SPACE_ID


In [0]:
from agent import AGENT

AGENT.predict({"messages": [{"role": "user", "content": "What is a lakehouse? Respond in one sentence !"}]})

🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: [REDACTED]
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****', 'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



/local_disk0/.ephemeral_nfs/envs/pythonEnv-76f81627-85a4-46c5-b7a9-35eb9f206793/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


ChatAgentResponse(messages=[ChatAgentMessage(role='assistant', content='A lakehouse is a data management architecture that combines the benefits of data warehouses and data lakes, allowing for the storage and analysis of both structured and unstructured data in a single, unified platform.', name=None, id='run--a310bf48-291f-44b0-b2a9-97aeba33a9ad-0', tool_calls=None, tool_call_id=None, attachments=None)], finish_reason=None, custom_outputs=None, usage=None)

Trace(request_id=tr-8e016a294d1f4492bff81a09303c3077)

In [0]:
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "What is 5+5 in python"}]}
):
    print(event, "-----------\n")

delta=ChatAgentMessage(role='assistant', content='', name=None, id='run--20e53362-4a4f-4234-829d-45113997682b-0', tool_calls=[ToolCall(id='call_d3861cd6-1897-4c13-aefb-02603c60a143', type='function', function=Function(name='system__ai__python_exec', arguments='{"code": "print(5+5)"}'))], tool_call_id=None, attachments=None) finish_reason=None custom_outputs=None usage=None -----------

delta=ChatAgentMessage(role='tool', content='{"format": "SCALAR", "value": "10\\n"}', name='system__ai__python_exec', id='2bfdc766-a264-4708-a905-32c137221a90', tool_calls=None, tool_call_id='call_d3861cd6-1897-4c13-aefb-02603c60a143', attachments=None) finish_reason=None custom_outputs=None usage=None -----------

delta=ChatAgentMessage(role='assistant', content='The result of 5+5 in Python is 10.', name=None, id='run--6bd851ae-09e3-495b-9192-5e1fafb5bb2a-0', tool_calls=None, tool_call_id=None, attachments=None) finish_reason=None custom_outputs=None usage=None -----------



Trace(request_id=tr-ab8a31b0b3cc44f0bf43317783108d49)

## Log the agent as an MLflow model

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

### Enable automatic authentication for Databricks resources
For the most common Databricks resource types, Databricks supports and recommends declaring resource dependencies for the agent upfront during logging. This enables automatic authentication passthrough when you deploy the agent. With automatic authentication passthrough, Databricks automatically provisions, rotates, and manages short-lived credentials to securely access these resource dependencies from within the agent endpoint.

To enable automatic authentication, specify the dependent Databricks resources when calling `mlflow.pyfunc.log_model().`

  - **TODO**: If your Unity Catalog tool queries a [vector search index](docs link) or leverages [external functions](docs link), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See docs ([AWS](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#specify-resources-for-automatic-authentication-passthrough) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#resources)).



In [0]:
import mlflow
from agent import tools, LLM_ENDPOINT_NAME
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool
from pkg_resources import get_distribution

model_config = mlflow.models.ModelConfig(development_config="chain_config.yaml")

resources = [DatabricksServingEndpoint(endpoint_name=model_config.get("LLM_ENDPOINT_NAME"))]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))


with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="agent.py",
        model_config="chain_config.yaml",
        extra_pip_requirements= [
            f"databricks-connect=={get_distribution('databricks-connect').version}",
            "arize-otel", "openinference.instrumentation.langchain"
            ],
        resources=resources,
    )

🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: [REDACTED]
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****', 'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



2025/05/23 23:25:58 INFO mlflow.pyfunc: Predicting on input example to validate output


🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: [REDACTED]
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****', 'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



2025/05/23 23:26:14 INFO mlflow.models.model: Found the following environment variables used during model inference: [ARIZE_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


Uploading artifacts:   0%|          | 0/13 [00:00<?, ?it/s]

## Evaluate the agent with Agent Evaluation

Use Mosaic AI Agent Evaluation to evalaute the agent's responses based on expected responses and other evaluation criteria. Use the evaluation criteria you specify to guide iterations, using MLflow to track the computed quality metrics.
See Databricks documentation ([AWS]((https://docs.databricks.com/aws/generative-ai/agent-evaluation) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-evaluation/)).


To evaluate your tool calls, add custom metrics. See Databricks documentation ([AWS](https://docs.databricks.com/en/generative-ai/agent-evaluation/custom-metrics.html#evaluating-tool-calls) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-evaluation/custom-metrics#evaluating-tool-calls)).

In [0]:
import pandas as pd

eval_examples = [
    {
        "request": {"messages": [{"role": "user", "content": "What is an 4*3 in python?"}]},
        "expected_response": None,
    }
]

eval_dataset = pd.DataFrame(eval_examples)
display(eval_dataset)


request,expected_response
"List(List(List(What is an 4*3 in python?, user)))",null


In [0]:
import mlflow

with mlflow.start_run(run_id=logged_agent_info.run_id):
    eval_results = mlflow.evaluate(
        f"runs:/{logged_agent_info.run_id}/agent",
        data=eval_dataset,  # Your evaluation dataset
        model_type="databricks-agent",  # Enable Mosaic AI Agent Evaluation
    )

# Review the evaluation results in the MLFLow UI (see console output), or access them in place:
eval_results.tables['eval_results'].head(10)

🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: [REDACTED]
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****', 'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



2025/05/23 23:32:02 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


Evaluating:   0%|          | 0/1 [Elapsed: 00:00, Remaining: ?]

{"ts": "2025-05-23 23:32:03,410", "level": "ERROR", "logger": "pyspark.sql.connect.client.logging", "msg": "GRPC Error received", "context": {}, "exception": {"class": "_MultiThreadedRendezvous", "msg": "<_MultiThreadedRendezvous of RPC that terminated with:\n\tstatus = StatusCode.FAILED_PRECONDITION\n\tdetails = \"BAD_REQUEST: session_id is no longer usable. Generate a new session_id by detaching and reattaching the compute and then try again [sessionId=267cd3f6-7c9b-45af-ba59-bcc78b41dbb3, reason=INACTIVITY_TIMEOUT]. (requestId=5e0e24bb-248c-42e4-b1e1-9f8b0c21ddce)\"\n\tdebug_error_string = \"UNKNOWN:Error received from peer  {created_time:\"2025-05-23T23:32:03.409275004+00:00\", grpc_status:9, grpc_message:\"BAD_REQUEST: session_id is no longer usable. Generate a new session_id by detaching and reattaching the compute and then try again [sessionId=267cd3f6-7c9b-45af-ba59-bcc78b41dbb3, reason=INACTIVITY_TIMEOUT]. (requestId=5e0e24bb-248c-42e4-b1e1-9f8b0c21ddce)\"}\"\n>", "stacktrace"

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results: 1/1 pass

,request_id,request,response,trace,tool_calls,response/overall_assessment/rating,response/llm_judged/safety/rating,response/llm_judged/safety/rationale,response/llm_judged/relevance_to_query/rating,response/llm_judged/relevance_to_query/rationale,agent/latency_seconds
0,ae1bdf509270aca8a0fa7f7928d52871a73a899eeb9b64...,"{'messages': [{'role': 'user', 'content': 'Wha...","{'messages': [{'role': 'assistant', 'content':...","{""info"": {""request_id"": ""tr-4741c0bd8a19459a81...","[{'tool_name': 'system__ai__python_exec', 'too...",yes,yes,No harmful content detected in response,yes,"The question asks about '4*3 in python,' which...",6.204


## Pre-deployment agent validation
Before registering and deploying the agent, perform pre-deployment checks using the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See Databricks documentation ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/model-serving-debug.html#validate-inputs) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/model-serving-debug#before-model-deployment-validation-checks)).

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={"messages": [{"role": "user", "content": "Hello!"}]},
    env_manager="uv",
)

2025/05/23 23:32:30 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/05/23 23:32:32 INFO mlflow.utils.virtualenv: Creating a new environment in /tmp/virtualenv_envs/mlflow-7a88de29fb97acfd3f7d6cf3719b44f730c7f95e with python version 3.10.12 using uv
Using CPython 3.10.12 interpreter at: /usr/bin/python3.10
Creating virtual environment at: /tmp/virtualenv_envs/mlflow-7a88de29fb97acfd3f7d6cf3719b44f730c7f95e
Activate with: source /tmp/virtualenv_envs/mlflow-7a88de29fb97acfd3f7d6cf3719b44f730c7f95e/bin/activate
2025/05/23 23:32:34 INFO mlflow.utils.virtualenv: Installing dependencies
Using Python 3.10.12 environment at: /tmp/virtualenv_envs/mlflow-7a88de29fb97acfd3f7d6cf3719b44f730c7f95e
Resolved 3 packages in 79ms
Prepared 3 packages in 258ms
Installed 3 packages in 24ms
 + pip==22.3.1
 + setuptools==80.8.0
 + wheel==0.38.4
Using Python 3.10.12 environment at: /tmp/virtualenv_envs/mlflow-7a88de29fb97acfd3f7d6cf3719b44f730c7f95e
Resolved 188 p

🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: [REDACTED]
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****', 'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

{"messages": [{"role": "assistant", "content": "Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", "id": "run--e0a3f175-221c-4fb3-bb3d-1dba4d733dd9-0"}]}

INFO:py4j.clientserver:Closing down clientserver connection


## Register the model to Unity Catalog

Before you deploy the agent, you must register the agent to Unity Catalog.

- **TODO** Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "prasad_kona_isv"
schema = "demo"
model_name = "langgraph-tool-calling-agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

Registered model 'prasad_kona_isv.demo.langgraph-tool-calling-agent' already exists. Creating a new version of this model...
Created version '3' of model 'prasad_kona_isv.demo.langgraph-tool-calling-agent'.


## Deploy the agent

In [0]:
from databricks import agents
agents.deploy(
                UC_MODEL_NAME, 
                uc_registered_model_info.version, 
                tags = {"endpointSource": "docs"},
                scale_to_zero_enabled=True,
                environment_vars={
                    "ARIZE_API_KEY": "{{secrets/prasad_kona/ARIZE_API_KEY}}",
                    "ARIZE_SPACE_ID": "{{secrets/prasad_kona/ARIZE_SPACE_ID}}",
                }
              )


    Deployment of prasad_kona_isv.demo.langgraph-tool-calling-agent version 3 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://e2-demo-field-eng.cloud.databricks.com/ml/endpoints/agents_prasad_kona_isv-demo-langgraph-tool-calling-agent
    Review App: https://e2-demo-field-eng.cloud.databricks.com/ml/review-v2/2ab25ac0f88b4234949a9bdd0d198d06/chat
    Monitor: https://e2-demo-field-eng.cloud.databricks.com/ml/experiments/913238801424870/evaluation-monitoring


Deployment(model_name='prasad_kona_isv.demo.langgraph-tool-calling-agent', model_version='3', endpoint_name='agents_prasad_kona_isv-demo-langgraph-tool-calling-agent', served_entity_name='prasad_kona_isv-demo-langgraph-tool-calling-agent_3', query_endpoint='https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints/agents_prasad_kona_isv-demo-langgraph-tool-calling-agent/served-models/prasad_kona_isv-demo-langgraph-tool-calling-agent_3/invocations', endpoint_url='https://e2-demo-field-eng.cloud.databricks.com/ml/endpoints/agents_prasad_kona_isv-demo-langgraph-tool-calling-agent', review_app_url='https://e2-demo-field-eng.cloud.databricks.com/ml/review-v2/2ab25ac0f88b4234949a9bdd0d198d06/chat')

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See Databricks documentation ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)).